<a href="https://colab.research.google.com/github/mostafa-ja/sample/blob/master/PyTorch_Tutorial_16_How_To_Use_The_TensorBoard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Youtube link](https://www.youtube.com/watch?v=VJW9wU-1n18&list=PLqnslRFeH2UrcDBWF5mfPGpqQDSta6VK4&index=16)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import matplotlib.pyplot as plt


In [ ]:
writer = SummaryWriter('runs/mnist1')

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# hyper parameters
input_size = 784
hidden_size = 500
num_classes = 10
batch_size = 4
num_epochs = 1
learning_rate = 0.001

# Load data

In [ ]:
data_transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                 torchvision.transforms.Normalize([0.1307,], [0.3081,])]) # just one element because of shape (1, 28, 28)

In [ ]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST('/content/data',train=True, transform=data_transform, download=True)
test_dataset = torchvision.datasets.MNIST('/content/data',train=False, transform=data_transform, download=True)

# Data loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=False)

In [ ]:
# samples

example_data,example_label = next(iter(train_loader))
img_grid = torchvision.utils.make_grid(example_data)


############## TENSORBOARD ########################
writer.add_image('samples',img_grid)

# writer.close()
# %tensorboard --logdir=runs

In [ ]:
print(example_data.shape)
print(img_grid.shape)

torch.Size([4, 1, 28, 28])
torch.Size([3, 32, 122])


In [ ]:
# Fully connected neural network with one hidden layer

class NeuralNet(nn.Module):

  def __init__(self,input_size,hidden_size,num_classes):
    super().__init__()
    self.linear1 = nn.Linear(input_size,hidden_size)
    self.relu = nn.ReLU()
    self.linear2 = nn.Linear(hidden_size,num_classes)

  def forward(self,x):
    out = self.linear1(x)
    out = self.relu(out)
    out = self.linear2(out)
    # no activation and no softmax at the end
    return out

In [ ]:
model = NeuralNet(input_size,hidden_size,num_classes)
model.to(device) # Set the model to run on the GPU.

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)


############## TENSORBOARD ########################
writer.add_graph(model,example_data.reshape(-1,28*28))

# writer.close()
# %tensorboard --logdir=runs


# Training the model

In [35]:
example_data.shape[0]

4

In [ ]:
running_loss = 0.0
running_correct = 0
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
  print(f'Epoch {epoch+1}/{num_epochs} ')
  print('_' * 10)

  for i,images,labels in enumerate(train_loader):
    # origin shape: [batch_s, 1, 28, 28] , resized: [100, 784]
    images = images.reshape(-1,28*28).to(device)
    labels = labels.to(device) # # Set the data to run on the GPU.

    outputs = model(images)
    loss = criterion(outputs,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()     # OR running_loss += loss.item()* images.size(0)   :  images.size(0) == images.shape[0] == batch_size

    _, predicted = torch.max(outputs.data, 1)
    running_correct += (labels==predicted).sum().item()

    if (i+1) % 100 == 0:
      print(f'Step {i+1}/{n_total_steps} , Loss : {loss.item():.4f}  ')

      ############## TENSORBOARD ########################
      
    




